In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# Repo root for src imports
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
except Exception:
    pass
def _find_repo_root():
    cwd = Path.cwd().resolve()
    for p in [Path('/content/drive/MyDrive/multihead-attention-robustness'),
              Path('/content/drive/My Drive/multihead-attention-robustness'),
              Path('/content/repo_run')]:
        if (p / 'src').exists():
            return p
    drive_root = Path('/content/drive')
    if drive_root.exists():
        for base in [drive_root / 'MyDrive', drive_root / 'My Drive', drive_root]:
            p = base / 'multihead-attention-robustness'
            if p.exists() and (p / 'src').exists():
                return p
    p = cwd
    for _ in range(10):
        if (p / 'src').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return cwd.parent if cwd.name == 'notebooks' else cwd
repo_root = _find_repo_root()
sys.path.insert(0, str(repo_root))
from src.models.feature_token_transformer import FeatureTokenTransformer, SingleHeadTransformer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
# Preserve models/training_history from prior notebooks (02, 03) when run in pipeline
if 'models' not in globals() or not isinstance(globals().get('models'), dict) or len(globals().get('models', {})) == 0:
    models = {}
if 'training_history' not in globals() or not isinstance(globals().get('training_history'), dict) or len(globals().get('training_history', {})) == 0:
    training_history = {}
TRAINING_CONFIG = {
    'ols': {}, 'ridge': {'alpha': 1.0},
    'mlp': {'hidden_dims': [128, 64], 'learning_rate': 0.001, 'batch_size': 64, 'epochs': 100, 'patience': 10},
    'transformer': {'d_model': 72, 'num_heads': 8, 'num_layers': 2, 'd_ff': 512, 'dropout': 0.1,
                   'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 100, 'patience': 20}
}


Mounted at /content/drive


In [ ]:
# Load fresh data from master_table.csv (standalone: each notebook pulls its own data)
data_path = repo_root / 'data' / 'cross_sectional' / 'master_table.csv'
df = pd.read_csv(data_path)
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
class CrossSectionalDataSplitter:
    def __init__(self, train_start='2005-01-01', train_end='2017-12-31', val_start='2018-01-01', val_end='2019-12-31'):
        self.train_start, self.train_end = train_start, train_end
        self.val_start, self.val_end = val_start, val_end
    def split(self, master_table):
        master_table = master_table.copy()
        master_table.index = pd.to_datetime(master_table.index)
        return {'train': master_table.loc[self.train_start:self.train_end], 'val': master_table.loc[self.val_start:self.val_end]}
    def prepare_features_labels(self, data):
        if data.empty:
            return pd.DataFrame(), pd.Series()
        numeric_data = data.select_dtypes(include=[np.number])
        if numeric_data.empty:
            return pd.DataFrame(), pd.Series()
        exclude_cols = ['mktcap', 'market_cap', 'date', 'year', 'month', 'ticker', 'permno', 'gvkey']
        target_cols = ['return', 'returns', 'ret', 'target', 'y', 'next_return', 'forward_return', 'ret_1', 'ret_1m', 'ret_12m', 'future_return', 'returns_1d']
        target_col = None
        for tc in target_cols:
            for col in numeric_data.columns:
                if tc.lower() in col.lower() and col.lower() not in [ec.lower() for ec in exclude_cols]:
                    target_col = col
                    break
            if target_col:
                break
        if target_col is None:
            potential = [c for c in numeric_data.columns if c.lower() not in [ec.lower() for ec in exclude_cols]]
            target_col = potential[-2] if len(potential) > 1 else (potential[-1] if potential else numeric_data.columns[-1])
        feature_cols = [c for c in numeric_data.columns if c != target_col and c.lower() not in [ec.lower() for ec in exclude_cols]]
        if not feature_cols:
            feature_cols = [c for c in numeric_data.columns if c != target_col]
        if not feature_cols:
            feature_cols = numeric_data.columns[:-1].tolist()
            target_col = numeric_data.columns[-1]
        return numeric_data[feature_cols], numeric_data[target_col]
splitter = CrossSectionalDataSplitter()
data_splits = splitter.split(df)
train_df, val_df = data_splits['train'], data_splits['val']
X_train_df, y_train = splitter.prepare_features_labels(train_df)
X_val_df, y_val = splitter.prepare_features_labels(val_df)
X_train = X_train_df.fillna(0).values.astype(np.float32)
y_train = y_train.fillna(0).values.astype(np.float32)
X_val = X_val_df.fillna(0).values.astype(np.float32)
y_val = y_val.fillna(0).values.astype(np.float32)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
print(f'Loaded fresh data: train {X_train_scaled.shape[0]}, val {X_val_scaled.shape[0]}')


Loaded fresh data: train 18826, val 3408


In [ ]:
"""
Train baseline models when models is empty (standalone notebook execution).
Run with: %run -i train_baseline_if_needed.py
Expects in namespace: X_train_scaled, y_train, X_val_scaled, y_val, device, TRAINING_CONFIG,
  RANDOM_SEED, FeatureTokenTransformer, SingleHeadTransformer, nn, torch, np,
  mean_squared_error, r2_score.
Populates: models, training_history.
"""
from sklearn.linear_model import LinearRegression, Ridge

def train_baseline_models():
    global models, training_history
    models = {}
    training_history = {}
    _device = globals().get('device', 'cpu')
    _cfg = globals().get('TRAINING_CONFIG', {})
    _seed = globals().get('RANDOM_SEED', 42)
    tr_cfg = _cfg.get('transformer', {'d_model': 72, 'num_heads': 8, 'num_layers': 2, 'd_ff': 512,
        'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': 100, 'patience': 20})

    def _train_transformer(model, X_train, y_train, X_val, y_val):
        model = model.to(_device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=tr_cfg['learning_rate'])
        X_t = torch.FloatTensor(X_train).to(_device)
        y_t = torch.FloatTensor(y_train).to(_device)
        X_v = torch.FloatTensor(X_val).to(_device)
        y_v = torch.FloatTensor(y_val).to(_device)
        nf = model.num_features if hasattr(model, 'num_features') else getattr(model.model, 'num_features', X_train.shape[1])
        if X_train.shape[1] != nf:
            if X_train.shape[1] < nf:
                pad_t = np.zeros((X_train.shape[0], nf - X_train.shape[1]))
                pad_v = np.zeros((X_val.shape[0], nf - X_val.shape[1]))
                X_t = torch.FloatTensor(np.hstack([X_train, pad_t])).to(_device)
                X_v = torch.FloatTensor(np.hstack([X_val, pad_v])).to(_device)
            else:
                X_t = torch.FloatTensor(X_train[:, :nf]).to(_device)
                X_v = torch.FloatTensor(X_val[:, :nf]).to(_device)
        bs = tr_cfg['batch_size']
        best = float('inf')
        pc = 0
        for ep in range(tr_cfg['epochs']):
            model.train()
            for i in range(0, len(X_t), bs):
                optimizer.zero_grad()
                out = model(X_t[i:i+bs])
                out = out[0] if isinstance(out, tuple) else out
                criterion(out.squeeze(), y_t[i:i+bs]).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            model.eval()
            with torch.no_grad():
                vp = model(X_v)
                vp = vp[0] if isinstance(vp, tuple) else vp
                vl = criterion(vp.squeeze(), y_v).item()
            if vl < best:
                best, pc = vl, 0
            else:
                pc += 1
                if pc >= tr_cfg['patience']:
                    break
        model.eval()
        with torch.no_grad():
            pred = model(X_v)
            pred = (pred[0] if isinstance(pred, tuple) else pred).squeeze().cpu().numpy()
        return pred

    # OLS
    m = LinearRegression()
    m.fit(X_train_scaled, y_train)
    p = m.predict(X_val_scaled)
    models['OLS'] = m
    training_history['OLS'] = {'rmse': np.sqrt(mean_squared_error(y_val, p)), 'r2': r2_score(y_val, p)}

    # Ridge
    m = Ridge(alpha=_cfg.get('ridge', {}).get('alpha', 1.0), random_state=_seed)
    m.fit(X_train_scaled, y_train)
    p = m.predict(X_val_scaled)
    models['Ridge'] = m
    training_history['Ridge'] = {'rmse': np.sqrt(mean_squared_error(y_val, p)), 'r2': r2_score(y_val, p)}

    # XGBoost
    try:
        import xgboost as xgb
        m = xgb.XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, subsample=0.8,
            colsample_bytree=0.8, random_state=_seed, objective='reg:squarederror', eval_metric='rmse')
        m.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], verbose=False)
        p = m.predict(X_val_scaled)
        models['XGBoost'] = m
        training_history['XGBoost'] = {'rmse': np.sqrt(mean_squared_error(y_val, p)), 'r2': r2_score(y_val, p)}
    except ImportError:
        pass

    # MLP
    class MLP(nn.Module):
        def __init__(self):
            super().__init__()
            h = _cfg.get('mlp', {}).get('hidden_dims', [128, 64])
            layers = []
            prev = X_train_scaled.shape[1]
            for d in h:
                layers += [nn.Linear(prev, d), nn.ReLU(), nn.Dropout(0.1)]
                prev = d
            layers.append(nn.Linear(prev, 1))
            self.net = nn.Sequential(*layers)
        def forward(self, x):
            return self.net(x).squeeze(-1)
    m = MLP().to(_device)
    opt = torch.optim.Adam(m.parameters(), lr=_cfg.get('mlp', {}).get('learning_rate', 0.001))
    Xt = torch.FloatTensor(X_train_scaled).to(_device)
    yt = torch.FloatTensor(y_train).to(_device)
    Xv = torch.FloatTensor(X_val_scaled).to(_device)
    for _ in range(min(_cfg.get('mlp', {}).get('epochs', 100), 50)):
        m.train()
        opt.zero_grad()
        torch.nn.functional.mse_loss(m(Xt), yt).backward()
        opt.step()
    m.eval()
    with torch.no_grad():
        p = m(Xv).cpu().numpy()
    models['MLP'] = m
    training_history['MLP'] = {'rmse': np.sqrt(mean_squared_error(y_val, p)), 'r2': r2_score(y_val, p)}

    # Transformers
    nf = X_train_scaled.shape[1]
    for name, cls, kw in [
        ('Single-Head', SingleHeadTransformer, {'num_features': nf, 'd_model': tr_cfg['d_model'], 'num_layers': tr_cfg['num_layers']}),
        ('Multi-Head', FeatureTokenTransformer, {'num_features': nf, 'd_model': tr_cfg['d_model'], 'num_heads': tr_cfg['num_heads'],
            'num_layers': tr_cfg['num_layers'], 'd_ff': tr_cfg['d_ff'], 'dropout': tr_cfg['dropout'], 'use_head_diversity': False}),
        ('Multi-Head Diversity', FeatureTokenTransformer, {'num_features': nf, 'd_model': tr_cfg['d_model'], 'num_heads': tr_cfg['num_heads'],
            'num_layers': tr_cfg['num_layers'], 'd_ff': tr_cfg['d_ff'], 'dropout': tr_cfg['dropout'],
            'use_head_diversity': True, 'diversity_weight': 0.01}),
    ]:
        mdl = cls(**kw)
        pred = _train_transformer(mdl, X_train_scaled, y_train, X_val_scaled, y_val)
        models[name] = mdl
        training_history[name] = {'rmse': np.sqrt(mean_squared_error(y_val, pred)), 'r2': r2_score(y_val, pred)}

    print(f"✓ Trained {len(models)} baseline models (standalone mode)")

if __name__ == '__main__':
    g = globals()
    if 'models' not in g or not isinstance(g.get('models'), dict) or len(g.get('models', {})) == 0:
        train_baseline_models()
    else:
        print("✓ Baseline models already loaded, skipping training")

✓ Trained 7 baseline models (standalone mode)


## Standard Training Summary

This notebook summarizes standard (clean) model performance and exports results to CSV.
Adversarial training is done in notebook 03.

In [ ]:
# Evaluate existing models under adversarial attacks
# This generates robustness_results and robustness_df without retraining

print("=" * 80)
print("EVALUATING EXISTING MODELS UNDER ADVERSARIAL ATTACKS")
print("=" * 80)

# Define attack epsilons and types
ATTACK_EPSILONS = [0.25, 0.5, 1.0]
ATTACK_TYPES = ['a1', 'a2', 'a3', 'a4']

# Attack functions (reuse from adversarial training section)
def apply_a1_attack(X, epsilon=0.01):
    """A1: Measurement Error - bounded perturbations."""
    noise = np.random.normal(0, epsilon, X.shape)
    return X + noise

def apply_a2_attack(X, missing_rate=0.1):
    """A2: Missingness/Staleness - set random features to zero."""
    X_adv = X.copy()
    n_samples, n_features = X.shape
    n_missing = max(1, int(n_features * missing_rate))
    for i in range(n_samples):
        missing_indices = np.random.choice(n_features, n_missing, replace=False)
        X_adv[i, missing_indices] = 0.0
    return X_adv

def apply_a3_attack(X, epsilon=0.01):
    """A3: Rank Manipulation - cross-sectional perturbation preserving ranks."""
    X_adv = X.copy()
    n_samples = X.shape[0]
    for i in range(n_samples):
        perturbation = np.random.normal(0, epsilon, X.shape[1])
        X_adv[i] = X[i] + perturbation
    return X_adv

def apply_a4_attack(X, epsilon=1.0):
    """A4: Regime Shift - distribution shift attack."""
    X_adv = X.copy()
    feature_std = np.std(X, axis=0, keepdims=True) + 1e-8
    noise = np.random.normal(0, epsilon, X.shape) * feature_std
    X_adv = X + noise
    return X_adv

# Function to evaluate a model under attack
def evaluate_model_under_attack(model, model_name, X_val, y_val, attack_type, epsilon,
                                device='cpu', is_sklearn=False, num_runs=5):
    """Evaluate a model under a specific attack."""
    # Set model to eval mode
    if not is_sklearn:
        model.eval()
        for module in model.modules():
            if isinstance(module, nn.Dropout):
                module.eval()

    # Make clean predictions
    if is_sklearn:
        y_pred_clean = model.predict(X_val)
    else:
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X_val).to(device)
            output = model(X_tensor)
            # Handle tuple returns (some models return (predictions, attention_weights))
            if isinstance(output, tuple):
                y_pred_tensor = output[0]
            else:
                y_pred_tensor = output
            y_pred_clean = y_pred_tensor.cpu().numpy().flatten()

    # Calculate clean RMSE
    clean_rmse = np.sqrt(mean_squared_error(y_val, y_pred_clean))

    # Run attack multiple times and average
    adv_rmses = []
    for run in range(num_runs):
        # Apply attack
        if attack_type == 'a1':
            X_adv = apply_a1_attack(X_val, epsilon=epsilon)
        elif attack_type == 'a2':
            # Convert epsilon to missing rate
            missing_rate = min(epsilon / 10.0, 0.8)
            X_adv = apply_a2_attack(X_val, missing_rate=missing_rate)
        elif attack_type == 'a3':
            X_adv = apply_a3_attack(X_val, epsilon=epsilon)
        elif attack_type == 'a4':
            X_adv = apply_a4_attack(X_val, epsilon=epsilon)
        else:
            X_adv = X_val.copy()

        # Make adversarial predictions
        if is_sklearn:
            y_pred_adv = model.predict(X_adv)
        else:
            with torch.no_grad():
                X_adv_tensor = torch.FloatTensor(X_adv).to(device)
                output_adv = model(X_adv_tensor)
                # Handle tuple returns (some models return (predictions, attention_weights))
                if isinstance(output_adv, tuple):
                    y_pred_adv_tensor = output_adv[0]
                else:
                    y_pred_adv_tensor = output_adv
                y_pred_adv = y_pred_adv_tensor.cpu().numpy().flatten()

        # Calculate adversarial RMSE
        adv_rmse = np.sqrt(mean_squared_error(y_val, y_pred_adv))
        adv_rmses.append(adv_rmse)

    # Average across runs
    avg_adv_rmse = np.mean(adv_rmses)
    delta_rmse = avg_adv_rmse - clean_rmse

    # Calculate robustness: min(1.0, 1 - (ΔRMSE / RMSE_clean))
    if clean_rmse > 0:
        robustness = min(1.0, 1.0 - (delta_rmse / clean_rmse))
    else:
        robustness = 1.0

    return {
        'clean_rmse': clean_rmse,
        'adv_rmse': avg_adv_rmse,
        'delta_rmse': delta_rmse,
        'robustness': robustness
    }

# Initialize results (always set so downstream CI/summary cells never fail)
robustness_results = []
robustness_df = pd.DataFrame()

# Check what models are available
print("\nAvailable models:")
if 'models' in locals():
    print(f"  Standard models: {list(models.keys())}")
else:
    print("  ⚠ No standard models found")
    models = {}

if 'adversarial_models' in locals():
    print(f"  Adversarially trained models: {len(adversarial_models)} models")
else:
    print("  ⚠ No adversarially trained models found")
    adversarial_models = {}

# Ensure we have validation data
if 'X_val_scaled' not in locals() or 'y_val' not in locals():
    print("\n⚠ X_val_scaled or y_val not found. Please run data loading and splitting cells first.")
    robustness_df = pd.DataFrame()
else:
    print(f"\nValidation set: {len(X_val_scaled)} samples, {X_val_scaled.shape[1]} features")

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}\n")

    # Evaluate standard models
    print("Evaluating standard models...")
    for model_name, model in models.items():
        print(f"  {model_name}...")
        is_sklearn = model_name in ['OLS', 'Ridge', 'XGBoost']

        for attack_type in ATTACK_TYPES:
            for epsilon in ATTACK_EPSILONS:
                try:
                    result = evaluate_model_under_attack(
                        model, model_name, X_val_scaled, y_val.values if hasattr(y_val, 'values') else y_val,
                        attack_type, epsilon, device=device, is_sklearn=is_sklearn, num_runs=5
                    )
                    robustness_results.append({
                        'model_name': model_name,
                        'attack_type': attack_type,
                        'epsilon': epsilon,
                        'clean_rmse': result['clean_rmse'],
                        'adv_rmse': result['adv_rmse'],
                        'delta_rmse': result['delta_rmse'],
                        'robustness': result['robustness'],
                        'training_type': 'standard'
                    })
                except Exception as e:
                    print(f"    ⚠ Error evaluating {model_name} under {attack_type} (ε={epsilon}): {e}")

    # Evaluate adversarially trained models
    if len(adversarial_models) > 0:
        print("\nEvaluating adversarially trained models...")
        for model_key, adv_model in adversarial_models.items():
            # Extract base model name and attack info from key
            # Format: "Multi-Head Diversity (A1, ε=0.25)"
            base_model = model_key.split('(')[0].strip()
            print(f"  {model_key}...")

            for attack_type in ATTACK_TYPES:
                for epsilon in ATTACK_EPSILONS:
                    try:
                        result = evaluate_model_under_attack(
                            adv_model, model_key, X_val_scaled, y_val.values if hasattr(y_val, 'values') else y_val,
                            attack_type, epsilon, device=device, is_sklearn=False, num_runs=5
                        )
                        robustness_results.append({
                            'model_name': model_key,
                            'attack_type': attack_type,
                            'epsilon': epsilon,
                            'clean_rmse': result['clean_rmse'],
                            'adv_rmse': result['adv_rmse'],
                            'delta_rmse': result['delta_rmse'],
                            'robustness': result['robustness'],
                            'training_type': 'adversarial'
                        })
                    except Exception as e:
                        print(f"    ⚠ Error evaluating {model_key} under {attack_type} (ε={epsilon}): {e}")

    # Create DataFrame
    if len(robustness_results) > 0:
        robustness_df = pd.DataFrame(robustness_results)
        print("\n" + "=" * 80)
        print("EVALUATION COMPLETE")
        print("=" * 80)
        print(f"\n✓ Generated {len(robustness_results)} robustness evaluations")
        print(f"✓ Created robustness_df with shape: {robustness_df.shape}")
        print(f"\nSummary by model:")
        for model_name in robustness_df['model_name'].unique():
            model_data = robustness_df[robustness_df['model_name'] == model_name]
            avg_robustness = model_data['robustness'].mean()
            print(f"  {model_name}: Average Robustness = {avg_robustness:.4f}")
    else:
        print("\n⚠ No robustness results generated. Check that models are available.")
        robustness_df = pd.DataFrame()

EVALUATING EXISTING MODELS UNDER ADVERSARIAL ATTACKS

Available models:
  Standard models: ['OLS', 'Ridge', 'XGBoost', 'MLP', 'Single-Head', 'Multi-Head', 'Multi-Head Diversity']
  ⚠ No adversarially trained models found

Validation set: 3408 samples, 22 features
Using device: cpu

Evaluating standard models...
  OLS...
  Ridge...
  XGBoost...
  MLP...
  Single-Head...
  Multi-Head...
  Multi-Head Diversity...

EVALUATION COMPLETE

✓ Generated 84 robustness evaluations
✓ Created robustness_df with shape: (84, 8)

Summary by model:
  OLS: Average Robustness = 0.8115
  Ridge: Average Robustness = 0.8508
  XGBoost: Average Robustness = 0.8717
  MLP: Average Robustness = 0.7476
  Single-Head: Average Robustness = 0.9288
  Multi-Head: Average Robustness = 0.9016
  Multi-Head Diversity: Average Robustness = 0.8848


In [ ]:
# Print results (run this cell only to view results without re-running evaluation)
if 'robustness_df' in globals() and len(robustness_df) > 0:
    print("=" * 80)
    print("ROBUSTNESS RESULTS (standard + adversarially trained)")
    print("=" * 80)
    print(robustness_df.to_string(index=False))
    # Export to CSV
    output_dir = repo_root / 'outputs'
    output_dir.mkdir(parents=True, exist_ok=True)
    csv_path = output_dir / 'robustness_results.csv'
    robustness_df.to_csv(csv_path, index=False)
    print(f"\n✓ Exported to {csv_path}")
else:
    print("⚠ robustness_df not found or empty. Run the evaluation cell above first.")

ROBUSTNESS RESULTS (standard + adversarially trained)
          model_name attack_type  epsilon  clean_rmse  adv_rmse  delta_rmse  robustness training_type
                 OLS          a1     0.25    0.017520  0.018411    0.000891    0.949139      standard
                 OLS          a1     0.50    0.017520  0.020571    0.003051    0.825859      standard
                 OLS          a1     1.00    0.017520  0.028058    0.010538    0.398483      standard
                 OLS          a2     0.25    0.017520  0.017858    0.000338    0.980703      standard
                 OLS          a2     0.50    0.017520  0.017807    0.000287    0.983601      standard
                 OLS          a2     1.00    0.017520  0.018252    0.000732    0.958226      standard
                 OLS          a3     0.25    0.017520  0.018314    0.000795    0.954648      standard
                 OLS          a3     0.50    0.017520  0.020570    0.003050    0.825925      standard
                 OLS        